In [32]:
import dspy
import openai
import os
from dotenv import load_dotenv
import datetime
load_dotenv()


True

In [33]:
import GoogleCalendar as gc

In [34]:
openai_api_key = os.getenv("OPENAI_API_KEY")
lm = dspy.LM(api_key = openai_api_key, model = "gpt-3.5-turbo")
dspy.settings.configure(lm=lm)

In [ ]:
class CalendarAgent(dspy.Signature):
    """
    make calendar events 
    1. assume all events are in UTC
    2. do not add characters to event dates, e.g. "Z" for UTC
    """
    event_details = dspy.InputField(desc="the details of the calendar event")
    current_time = dspy.InputField(desc="the current time")

    event_summary  = dspy.OutputField(desc="the summary of the event")
    event_start = dspy.OutputField(desc="the start time of the event in iso format. utc")
    event_end = dspy.OutputField(desc="the end time of the event in iso format. utc")
    event_location = dspy.OutputField(desc="the location of the event")
    event_description = dspy.OutputField(desc="the description of the event")

In [36]:
class GetEvents(dspy.Signature):
    """
    from the given details make a list of events
    """

    details = dspy.InputField(desc="the details of the events")
    events: list[str] = dspy.OutputField(desc="the list of events")

In [37]:
class Assistant(dspy.Signature):
    """an assistant that can help with tasks. determine the best course of action. do not assume dates"""
    prompt = dspy.InputField(desc="given instructions to follow")
    output = dspy.OutputField(desc="the response to instructions")

In [38]:
def get_upcoming_events():
    """
    Get a list of upcoming events from the calendar.
    """
    client = gc.create_calendar_client()
    events = gc.get_events(client)
    return events

In [39]:
def create_calendar_event(users_input):
    """
    1. pass the users input to the agent.
    2. do not add characters to event dates, e.g. "Z" for UTC
    3. assume all events are upcoming.
    """
    calendar_agent = dspy.ReAct(
        signature=CalendarAgent,
        tools=[]
    )

    event_agent = dspy.Predict(signature = GetEvents)
    event_result = event_agent(details=users_input)
    

    client = gc.create_calendar_client()
    current_time = gc.get_current_time()

    print(f"Event list: {event_result.events}")
    print(f"Current time: {current_time}")

    for event in event_result.events:
        agent_response = calendar_agent(event_details= event, current_time=current_time)

        print(f"\n\nAgent response: \n{agent_response}")

        iso_start_time = datetime.datetime.fromisoformat(agent_response.event_start)
        iso_end_time = datetime.datetime.fromisoformat(agent_response.event_end)
        print(f"\nISO Start Time: {iso_start_time}")


        event = gc.create_event(
            service=client,
            summary=agent_response.event_summary,
            start_time=iso_start_time,
            end_time=iso_end_time,
            location=agent_response.event_location,
            description=agent_response.event_description
        )

In [40]:
react = dspy.ReAct(
    signature=Assistant,
    tools=[create_calendar_event, gc.get_current_time, get_upcoming_events]
)

In [44]:
response = react(prompt="what is the capitol of france?")
print(f"\n{response.output}")


Paris
